code from Feb 28 
currently having issues with subprocess check_output returning non-zero exit status when creating the html


###  How to open a file dialog box
Tkinter is the easiest way if you don't want to have any 
other dependencies. 
An alternative that uses tkinter is 
https://easygui.readthedocs.io/en/master/


```python
# open file dialog box
import tkinter as tk
from tkinter import filedialog

# To show only the dialog without any 
# other GUI elements, you have to hide the root window 
# using the withdraw method:
root = tk.Tk()
root.withdraw()

print("Select the Jupyter Notebook you wish to convert")
fn = filedialog.askopenfilename()
print(file_path)
```

### How to run a shell command from python
```python
# run a shell command from python and get the response
# as a string into a varaiable which you can print or discard
from subprocess import check_output
response = check_output("dir C:", shell=True)
print (response)
```

#### How to convert a Jupyter Notebook to HTML using nbconvert
##### verify that you have nbconvert installed as part of Anaconda
```cmd
jupyter nbconvert --help
```

The simplest way to use nbconvert is
    > jupyter nbconvert mynotebook.ipynb
which will convert mynotebook.ipynb to the default format (probably HTML).

You can specify the export format with `--to`.
Options include ['asciidoc', 'custom', 'html', 'latex', 'markdown', 'notebook', 'pdf', 'python', 'rst', 'script', 'slides']

In [ ]:
import os
# open file dialog box
import tkinter as tk
from tkinter import filedialog
import subprocess
# from subprocess import check_output

root = tk.Tk()
root.wm_attributes('-topmost', 1)
root.lift()
root.withdraw()

print("Select the Jupyter Notebook you wish to convert")
fn = filedialog.askopenfilename()
# print(file_path)
# jupyter nbconvert --to html --template basic Jupyter_Book_Notes.ipynb
print (fn)
cmd = "jupyter nbconvert --to html --template basic {0}".format(fn)
print (cmd)

os.system(cmd)

# try:
    # response = check_output(cmd, shell=True)
    # response = check_output(['jupyter','nbconvert --to html --template basic ' + fn])
    # print(response)
# except subprocess.CalledProcessError as e:
#    print (e)
    


In [2]:
# loop through a collection of notebooks and md files
import os
import tkinter as tk
from tkinter import filedialog

# set properties for the directory open dialog box
root = tk.Tk()
root.wm_attributes('-topmost', 1)
root.lift()
root.withdraw()

# for OSX
# self.root.lift()
# self.root.call('wm', 'attributes', '.', '-topmost', True)
# self.root.after_idle(self.root.call, 'wm', 'attributes', '.', '-topmost', False)


# create TOC (use directory structure and filenames)
def appendTOC(TOC,level,text):
    fh=open(TOC, "a")
    if level == 0:
        fh.write("{0}\n".format(text))
    if level == 1:
        fh.write("\t{0}\n".format(text))
    if level == 2:
        fh.write("\t\t{0}\n".format(text))
    fh.close()
    
    
print("Select the Directory containing Jupyter Notebooks you wish to convert")
path = filedialog.askdirectory()


for root, directories, filenames in os.walk(path):
    TOC = os.path.join(root,"TableOfContents")
    appendTOC(TOC,0,root)
    
    #for directory in directories:
    for filename in filenames:
        name, ext = os.path.splitext(filename)
        infile = os.path.join(root,filename)
        if ext == ".html":
            os.unlink(infile)
        else:
            outfile = os.path.join(path,filename)
            outfile += ".html"
            print(outfile)
        
            
            if filename.endswith("ipynb"):    
                cmd = "jupyter nbconvert --output \"{0}\" --to html --template basic \"{1}\"".format(outfile,infile)
                # print (cmd)
                os.system(cmd)                
                # add filename to TOC
                print("TOC: {0} {1}",TOC,filename)
                appendTOC(TOC,0,filename)
            
            if filename.endswith("md"):
                cmd = "grip \"{0}\" --export \"{1}\"".format(infile,outfile)
                # print (cmd)
                os.system(cmd)
                # add filename to TOC
                print("TOC: {0} {1}",TOC,filename)
                appendTOC(TOC,0,filename)
            
print ("done printing")


#create PDFs

# push to github


                  

Select the Directory containing Jupyter Notebooks you wish to convert
C:/Temp/aaa/Spring Boot\Lesson 32_5.md.html
TOC: {0} {1} C:/Temp/aaa/Spring Boot\IntroToSpringBoot\TableOfContents Lesson 32_5.md
C:/Temp/aaa/Spring Boot\Lesson 34 - Creating password reset functionality.html
C:/Temp/aaa/Spring Boot\Lesson00.md.html
TOC: {0} {1} C:/Temp/aaa/Spring Boot\IntroToSpringBoot\TableOfContents Lesson00.md
C:/Temp/aaa/Spring Boot\Lesson01.md.html
TOC: {0} {1} C:/Temp/aaa/Spring Boot\IntroToSpringBoot\TableOfContents Lesson01.md
C:/Temp/aaa/Spring Boot\Lesson02.md.html
TOC: {0} {1} C:/Temp/aaa/Spring Boot\IntroToSpringBoot\TableOfContents Lesson02.md
C:/Temp/aaa/Spring Boot\Lesson03.md.html
TOC: {0} {1} C:/Temp/aaa/Spring Boot\IntroToSpringBoot\TableOfContents Lesson03.md
C:/Temp/aaa/Spring Boot\Lesson03_5.md.html
TOC: {0} {1} C:/Temp/aaa/Spring Boot\IntroToSpringBoot\TableOfContents Lesson03_5.md
C:/Temp/aaa/Spring Boot\Lesson04.md.html
TOC: {0} {1} C:/Temp/aaa/Spring Boot\IntroToSpringBoot\T

In [10]:
# Generating the TOC
# Use the directory structure to generate the TOC.... it's easier than going through and 
# looking for tags - especially in JSON. 
# THe code below is an example of an atttempt to search the files using regex.

import sys
import re
import getopt

def generateTOC(filename):
    pattern = "#"
    bufsize = 64 * 1024
    reflags = 0 | re.IGNORECASE

    try:
        prog = re.compile(pattern, reflags)
    except re.error as msg:
        usage("error in regular expression: %s" % msg)


    try:
        f = open(filename)
    except IOError as msg:
        usage("can't open %r: %s" % (filename, msg), 1)

    f.seek(0, 2)
    pos = f.tell()
    leftover = None
    while pos > 0:
        size = min(pos, bufsize)
        pos = pos - size
        f.seek(pos)
        buffer = f.read(size)
        lines = buffer.split("\n")
        del buffer
        if leftover is None:
            if not lines[-1]:
                del lines[-1]
        else:
            lines[-1] = lines[-1] + leftover
        if pos > 0:
            leftover = lines[0]
            del lines[0]
        else:
            leftover = None
        for line in reversed(lines):
            if prog.search(line):
                print(line)


                
fn = r'C:\Temp\aaa\My Book\content\Chapter 2\Section 1\Python+Fundamentals.ipynb'
generateTOC(fn)                

    "### Comparisons\n",
    "# can you figure out the difference between = and ==\n",
    "# Printing\n"
    "    word = movielist[index].replace(\"\\n\", \"\") #remove line ending\n",
    "#---------------------------------------------------------------------------------\n",
    "#What directory is my file in???\n",
    "#It is not necessary to close the file\n",
    "#now open the file and read it into a dictionary\n",
    "        #print(\"You entered \" + answer)\n",
    "history = open(\"myfile.txt\",'a')#use w for write, a for append, r for read\n",
    "#---------------------------------------------------------------------------------\n",
    "#---------------------------------------------------------------------------------\n",
    "#---------------------------------------------------------------------------------\n",
    "# A python helper function that prints code \n",
    "#---------------------------------------------------------------------------------\n"
    "#----------

### Random notes

must install python grip... add notes explaining this

https://nbconvert.readthedocs.io/en/latest/index.html

https://www.blog.pythonlibrary.org/2018/10/09/how-to-export-jupyter-notebooks-into-other-formats/